In [1]:
from pathlib import Path
import pandas as pd
import pyspark
import dxpy
import hail as hl
from datetime import datetime
from matrixtables import import_mt, interval_qc_mt, smart_split_multi_mt
from subprocess import run

from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
output_notebook()

Path("/tmp").resolve().mkdir(parents=True, exist_ok=True)


Loading BokehJS ...

In [2]:
# Spark and Hail
VCF_DIR = Path("/mnt/project/Bulk/Exome sequences/Population level exome OQFE variants, pVCF format - final release/")
DATABASE = "matrix_tables"
REFERENCE_GENOME = 'GRCh38'

LOG_FILE = (
    Path("../hail_logs", f"GIPR_{datetime.now().strftime('%H%M')}.log")
    .resolve()
    .__str__()
)

sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession(sc)

try:
    mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]
except Exception as e:
    spark.sql(f"CREATE DATABASE {DATABASE} LOCATION  'dnax://'")
    mt_database = dxpy.find_one_data_object(name=DATABASE)["id"]

hl.init(sc=sc, default_reference=REFERENCE_GENOME, log=LOG_FILE)

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 2.4.4
SparkUI available at http://ip-10-60-131-24.eu-west-2.compute.internal:8081
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.78-b17627756568
LOGGING: writing to /opt/notebooks/gogoGPCR/notebooks/hail_logs/GIPR_1155.log


In [3]:
# Read in metadata and region
MAPPING_FILE = Path("../../data/misc/mappings_with_blocks.tsv").resolve()
mapping = pd.read_csv(MAPPING_FILE, sep="\t").set_index("HGNC", drop=False)
mapping.loc["GIPR",:]

HGNC                                                                     GIPR
entry_name                                                         gipr_human
name                                                             GIP receptor
accession                                                              P48546
family                                                        002_001_003_002
species                                                          Homo sapiens
residue_numbering_scheme                                            GPCRdb(B)
sequence                    MTTSPILQLLLRLSLCGLLLQRAETGSKGQTAGELYQRWERYRREC...
genes                                                                ['GIPR']
ENSG                                                          ENSG00000010310
ENST                                                          ENST00000590918
type                                                                     GPCR
GRCh37_start                                                    

In [4]:
# Import VCF as matrix table and filter to only GIPR
mt = import_mt(["GIPR"], mapping, vcf_dir=VCF_DIR, vcf_version="v1").key_rows_by(
    "locus", "alleles"
)

In [5]:
# Checkpoint because Hail likes checkpointing
stage = "RAW"
checkpoint_file = f"/tmp/GIPR.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)

v, s = mt.count()
print(f"{v} variants and {s} samples after import")

2023-01-31 11:02:04 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:16 Hail: INFO: wrote matrix table with 1214 rows and 469835 columns in 1 partition to /tmp/GIPR.RAW.cp.mt


1214 variants and 469835 samples after import


In [6]:
# Filter to only WES target regions 
INTERVAL_FILE=Path("../../data/misc/xgen_plus_spikein.b38.bed").resolve()
run(["hadoop", "fs", "-put", str(INTERVAL_FILE), "/tmp"])

interval_table = hl.import_bed(
        f"/tmp/{INTERVAL_FILE.name}",
        reference_genome="GRCh38",
    )

mt = mt.filter_rows(hl.is_defined(interval_table[mt.locus]))
print(f"{mt.count_rows()} variants after interval filtering")

2023-01-31 11:10:18 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)
2023-01-31 11:10:20 Hail: INFO: Coerced sorted dataset


489 variants after interval filtering


In [7]:
# Split multi alleles 
mt = mt.filter_rows(mt.alleles.length() <= 6)
mt = smart_split_multi_mt(mt)

print(f"{mt.count_rows()} variants with not more than 6 alleles after splitting")

2023-01-31 11:10:23 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:25 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:27 Hail: INFO: Coerced sorted dataset


555 variants with not more than 6 alleles after splitting


In [8]:
# Annotate with VEP and generate protein consequence
VEP_JSON = Path("../../data/misc/GRCh38_VEP.json").resolve()

mt = hl.vep(mt, f"file:{VEP_JSON}")

is_MANE = mt.aggregate_rows(
    hl.agg.all(hl.is_defined(mt.vep.transcript_consequences.mane_select))
)
assert is_MANE, "Selected transcript may not be MANE Select. Check manually."

mt = mt.annotate_rows(
    protCons=mt.vep.transcript_consequences.amino_acids[0].split("/")[0]
    + hl.str(mt.vep.transcript_consequences.protein_end[0])
    + mt.vep.transcript_consequences.amino_acids[0].split("/")[-1]
)

2023-01-31 11:10:29 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:30 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:31 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-01-31 11:10:32 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:34 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:35 Hail: INFO: Coerced sorted dataset
2023-01-31 11:10:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-01-31 11:10:37 Hail: INFO: Coerced sorted dataset


In [9]:
# Annotate variants of interest
interesting_variants = hl.literal(["missense_variant", "stop_gained", "frameshift_variant", "inframe_deletion", "start_lost"])

mt = mt.annotate_rows(is_interesting_var = interesting_variants.contains(mt.vep.most_severe_consequence))

In [10]:
# Load in vitro data
INVITRO_FILE="/opt/notebooks/gogoGPCR/data/GIPR/GIPR.invitro.csv"
run(["hadoop", "fs", "-put", str(INVITRO_FILE), "/tmp"])

ht = hl.import_table("/tmp/GIPR.invitro.csv", delimiter = ",", impute = True)
ht = ht.filter(ht.GRCh_38_location != "")
ht = ht.annotate(variant = "chr" + ht.GRCh_38_location.replace("-", ":"))
ht = ht.annotate(**hl.parse_variant(ht.variant))
ht = ht.key_by(ht.locus, ht.alleles)
ht.export("/tmp/GIPR/GIPR.invitro.tsv")
ht.write("/tmp/GIPR.invitro.ht")

run(["hadoop", "fs", "-get", "/tmp/GIPR/GIPR.invitro.tsv", "/opt/notebooks/gogoGPCR/data/GIPR/GIPR.invitro.tsv"])

2023-01-31 11:10:57 Hail: WARN: Found 1 duplicate column. Mangled columns follows:
  'Fmut(EC50)' -> 'Fmut(EC50)_1'
2023-01-31 11:10:57 Hail: INFO: Reading table to impute column types
2023-01-31 11:10:58 Hail: WARN: Found 1 duplicate column. Mangled columns follows:
  'Fmut(EC50)' -> 'Fmut(EC50)_1'
2023-01-31 11:10:58 Hail: INFO: Finished type imputation
  Loading field '\ufeffAposA' as type str (imputed)
  Loading field 'Location' as type str (imputed)
  Loading field 'Bmax (Binding)' as type str (imputed)
  Loading field 'sem (Bmax)' as type str (imputed)
  Loading field 'logIC50 (Binding)' as type str (imputed)
  Loading field 'sem (IC50)' as type str (imputed)
  Loading field 'n (binding)' as type str (imputed)
  Loading field 'Fmut (IC50)' as type str (imputed)
  Loading field 'Emax (cAMP)' as type str (imputed)
  Loading field 'sem (EC50)' as type str (imputed)
  Loading field 'logEC50 (cAMP)' as type str (imputed)
  Loading field 'sem (cAMP)' as type str (imputed)
  Loading fie

CompletedProcess(args=['hadoop', 'fs', '-get', '/tmp/GIPR/GIPR.invitro.tsv', '/opt/notebooks/gogoGPCR/data/GIPR/GIPR.invitro.tsv'], returncode=1)

In [11]:
# Annotate with in vitro data
mt = mt.annotate_rows(**ht[mt.locus, mt.alleles])

In [12]:
# SCORE_FILE=https://krishna.gs.washington.edu/download/CADD/v1.6/GRCh38/whole_genome_SNVs.tsv.gz
# INDEX=IndexFile
# curl -o $INDEX $SCORE_FILE.tbi
# tabix $SCORE_FILE $INDEX --print-header 19:45668221-45683722 | tail -n +2 > ../../data/GIPR/GIPR.CADD.tsv

In [13]:
# Import CADD scores
run(["hadoop", "fs", "-put", "../../data/GIPR/GIPR.CADD.tsv", "/tmp/GIPR.CADD.tsv"])
cadd = hl.import_table("/tmp/GIPR.CADD.tsv", impute = True)
cadd = cadd.annotate(variant_string = "chr" + hl.str(cadd.Chrom) + ":" + hl.str(cadd.Pos) + ":" + cadd.Ref + ":" + cadd.Alt)
cadd = cadd.annotate(**hl.parse_variant(cadd.variant_string))
cadd = cadd.key_by(cadd.locus, cadd.alleles)

2023-01-31 11:11:03 Hail: INFO: Reading table to impute column types
2023-01-31 11:11:04 Hail: INFO: Finished type imputation
  Loading field 'Chrom' as type int32 (imputed)
  Loading field 'Pos' as type int32 (imputed)
  Loading field 'Ref' as type str (imputed)
  Loading field 'Alt' as type str (imputed)
  Loading field 'RawScore' as type float64 (imputed)
  Loading field 'PHRED' as type float64 (imputed)


In [14]:
# Annotate with CADD data
mt = mt.annotate_rows(**cadd[mt.locus, mt.alleles])

In [15]:
# Checkpoint because Hail likes checkpointing
stage = "ANNOTATED"
checkpoint_file = f"/tmp/GIPR.{stage}.cp.mt"

mt = mt.checkpoint(checkpoint_file, overwrite=True)
# mt = hl.read_matrix_table(checkpoint_file)

mt.filter_rows(~hl.is_missing(mt['Bmax (Binding)'])).count_rows() # 32

2023-01-31 11:11:05 Hail: INFO: Coerced sorted dataset
2023-01-31 11:11:06 Hail: INFO: Coerced sorted dataset
2023-01-31 11:12:25 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-01-31 11:12:26 Hail: INFO: Coerced sorted dataset
2023-01-31 11:12:26 Hail: INFO: Coerced sorted dataset
2023-01-31 11:12:27 Hail: INFO: Coerced sorted dataset
2023-01-31 11:16:56 Hail: INFO: wrote matrix table with 555 rows and 469835 columns in 3 partitions to /tmp/GIPR.ANNOTATED.cp.mt
    Total size: 1.32 GiB
    * Rows/entries: 1.32 GiB
    * Columns: 3.07 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (20.00 B)
    * Largest partition:  483 rows (1.14 GiB)


32

In [16]:
# Filter oligo lots
mt.count_rows()
mt = mt.annotate_rows(DP10 = hl.agg.mean(mt.DP < 10)) # UKB recommend
mt = mt.filter_rows(mt.DP10 < 0.90) # No variants
mt.count_rows()

555

In [17]:
# Entries QC
mt = mt.annotate_entries(AB=(mt.AD[1] / hl.sum(mt.AD)))

mt = mt.filter_entries(
    (mt.GQ >= 20) &
    (
        hl.is_indel(mt.alleles[0], mt.alleles[1]) & (mt.DP >= 10)) |
        (hl.is_snp(mt.alleles[0], mt.alleles[1]) & (mt.DP >= 7)
    ) &
    (
        (mt.GT.is_hom_ref() & (mt.AB <= 0.1)) |
        (mt.GT.is_het() & (mt.AB >= 0.2) & (mt.AB <= 0.8)) |
        (mt.GT.is_hom_var() & (mt.AB >= 0.9))
    )
) # Combine Backman and Pedersen

mt = mt.compute_entry_filter_stats()

In [18]:
# Time for variant and sample QC
mt = hl.variant_qc(mt)
mt = hl.sample_qc(mt)

In [5]:
# Checkpoint QC
stage = "QC0"
checkpoint_file = f"/tmp/GIPR.{stage}.cp.mt"

# mt = mt.checkpoint(checkpoint_file, overwrite=True)
mt = hl.read_matrix_table(checkpoint_file)

In [6]:
# Variant QC
mt = mt.filter_rows(~(mt.variant_qc.p_value_hwe < 10**-15)) # Backman, 2 variants, chr19:45674703 ["C","T"] "N170N" and chr19:45677071 ["G","C"]"E252D"
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.99) # Backman
mt = mt.filter_rows(~hl.is_missing(mt.variant_qc.AF)) # 10 variants, no missense
mt.count_rows()

542

In [7]:
# Sample QC
mt = mt.filter_cols(mt.sample_qc. call_rate > 0.9) # 1 sample
mt = mt.filter_cols(~mt.s.startswith("W"))

print(f"Samples remaining after removing withdrawn participants: {mt.count_cols()} ")

Samples remaining after removing withdrawn participants: 469817 


In [8]:
# Ancestry check
run(["hadoop", "fs", "-put", "ancestry.csv", "/tmp"]) # from GIPR_ancestry.ipynb
ht = hl.import_table("/tmp/ancestry.csv", delimiter = ",", quote='"', missing="foo").select("PC_UKBB.eid", "group").key_by("PC_UKBB.eid")

2023-01-31 11:56:27 Hail: INFO: Reading table without type imputation
  Loading field '' as type str (not specified)
  Loading field 'PC_UKBB.eid' as type str (not specified)
  Loading field 'group' as type str (not specified)


In [9]:
mt = mt.annotate_cols(**ht[mt.s])

In [10]:
mt2 = mt.filter_rows(~hl.is_missing(mt['Bmax (Binding)']))
mt2 = mt2.annotate_cols(carrier = hl.agg.any(mt2.GT.is_non_ref()))
grp = mt2.select_cols(mt2.group, mt2.carrier).cols()
grp = grp.group_by(grp.group).aggregate(num_carriers = hl.agg.sum(grp.carrier))
grp.show(-1)
grp.export('/tmp/n_ancestries.csv')
run(["hadoop", "fs", "-get", "/tmp/n_ancestries.csv"])

2023-01-31 11:56:27 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'
2023-01-31 11:56:59 Hail: INFO: Ordering unsorted dataset with network shuffle


,
group,num_carriers
str,int64
"""Ashkenazi""",22
"""Caribbean""",108
"""China""",34
"""India""",110
"""Iran""",18
"""Italy""",111
"""NA""",206
"""Nigeria""",155


2023-01-31 11:57:28 Hail: INFO: Ordering unsorted dataset with network shuffle
2023-01-31 11:57:28 Hail: INFO: merging 11 files totalling 139...
2023-01-31 11:57:29 Hail: INFO: while writing:
    /tmp/n_ancestries.csv
  merge time: 100.696ms


CompletedProcess(args=['hadoop', 'fs', '-get', '/tmp/n_ancestries.csv'], returncode=1)

In [11]:
# Filter only United Kingdom
# mt = mt.filter_cols(mt.group == "United Kingdom")
mt.count_cols()

469817

In [12]:
# Checkpoint QC
stage = "QC1"
checkpoint_file = f"/tmp/GIPR.{stage}.cp.mt"

# mt = mt.checkpoint(checkpoint_file, overwrite=True)
# mt = hl.read_matrix_table(checkpoint_file)

In [13]:
# Remove filtered samples
s2r = hl.import_table("/tmp/samples_to_remove.tsv").key_by('eid')
mt = mt.anti_join_cols(s2r)
mt.count_cols()

2023-01-31 11:57:31 Hail: INFO: Reading table without type imputation
  Loading field 'eid' as type str (not specified)


465506

In [14]:
# Remove variants no longer present
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.n_non_ref != 0)

In [15]:
mt.count_rows()

535

In [16]:
# ANNOTATION
mt = mt.annotate_rows(eff_mask = mt["100pM efficacy"].replace("WT-like", "WT"),
                      arr_mask = mt.Arrestin.replace("WT-like", "WT"))
mt = mt.annotate_rows(label0 = mt.eff_mask + "_" + mt.arr_mask)

In [17]:
# pLoF variant counts
mt.aggregate_rows(hl.agg.counter(mt.vep.most_severe_consequence))

frozendict({'frameshift_variant': 16, 'inframe_deletion': 5, 'inframe_insertion': 2, 'missense_variant': 340, 'splice_region_variant': 5, 'start_lost': 2, 'stop_gained': 27, 'synonymous_variant': 138})

In [18]:
# Create label
pLoF = hl.set(["stop_gained", "frameshift_variant", "splice_region_variant", "start_lost"])

mt = mt.annotate_rows(label = hl.case()
                         .when(~hl.is_missing(mt.label0), mt.label0)
                         .when(pLoF.contains(mt.vep.most_severe_consequence) | (mt.PHRED > 30), 'pLoF')
                         .when((mt.vep.most_severe_consequence == 'synonymous_variant') | (mt.PHRED < 10), 'Synonymous')
                         .default(mt.vep.most_severe_consequence)
#                          .or_missing()
                )

mt.label.show(10)
mt.aggregate_rows(hl.agg.counter(mt.label))

,,
locus,alleles,label
locus<GRCh38>,array<str>,str
chr19:45669521,"[""A"",""ATGACTACCT""]",NA
chr19:45669521,"[""A"",""G""]","""pLoF"""
chr19:45669521,"[""A"",""T""]","""pLoF"""
chr19:45669554,"[""C"",""G""]","""missense_variant"""
chr19:45669557,"[""C"",""T""]","""WT_WT"""
chr19:45669559,"[""C"",""G""]","""Synonymous"""
chr19:45669565,"[""G"",""T""]","""Synonymous"""
chr19:45669568,"[""C"",""A""]","""pLoF"""


frozendict({'GoF_WT': 1, 'LoF_LoF': 15, 'LoF_LoF ': 1, 'LoF_WT': 4, 'Synonymous': 163, 'WT_GoF': 1, 'WT_LoF': 4, 'WT_WT': 6, 'missense_variant': 230, 'pLoF': 103, None: 7})

In [20]:
# Get summary of variants
intr = mt.rows()
intr = intr.select(intr.variant_qc, intr.protCons, intr.label, intr.vep.most_severe_consequence, intr.PHRED)
intr = intr.annotate(**intr.variant_qc)
intr = intr.drop(
    "variant_qc",
    "gq_stats",
    "dp_stats",
)
intr.export('/tmp/GIPR_new_variants_ALL.tsv')
run(["hadoop", "fs", "-get", '/tmp/GIPR_new_variants_ALL.tsv'])

2023-01-31 12:04:10 Hail: INFO: merging 2 files totalling 100.5K...
2023-01-31 12:04:10 Hail: INFO: while writing:
    /tmp/GIPR_new_variants_ALL.tsv
  merge time: 33.103ms


CompletedProcess(args=['hadoop', 'fs', '-get', '/tmp/GIPR_new_variants_ALL.tsv'], returncode=0)

In [21]:
# Generate regenioe files
from matrixtables import write_bgen, add_varid

In [22]:
BGEN_FILE = Path("/opt/notebooks/gogoGPCR/data/GIPR/GIPR").resolve().__str__()
ANNOTATIONS_FILE = Path("/opt/notebooks/gogoGPCR/data/GIPR/GIPR.annotations").resolve().__str__()
SETLIST_FILE = Path("/opt/notebooks/gogoGPCR/data/GIPR/GIPR.setlist").resolve().__str__()

In [24]:
# .bgen and .sample files
write_bgen(mt, "file:" + BGEN_FILE)

2023-01-31 12:07:36 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/data/GIPR/GIPR.bgen
  merge time: 172.190ms
2023-01-31 12:08:31 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/data/GIPR/GIPR.bgen
  merge time: 161.490ms


In [25]:
# .annotations file

mt = add_varid(mt)

annotations = (
    mt.select_rows(
        varid=mt.varid,
        gene=mt.vep.transcript_consequences.gene_symbol[0],
        annotation=mt.label,
    )
    .rows()
    .key_by("varid")
    .drop("locus")
    .drop("alleles")
)

annotations.export("file:" + ANNOTATIONS_FILE, header=False)


2023-01-31 12:09:01 Hail: INFO: Coerced sorted dataset
2023-01-31 12:09:29 Hail: INFO: merging 2 files totalling 19.0K...
2023-01-31 12:09:29 Hail: INFO: while writing:
    file:/opt/notebooks/gogoGPCR/data/GIPR/GIPR.annotations
  merge time: 20.012ms


In [26]:
# .setlist file
position = mt.aggregate_rows(hl.agg.min(mt.locus.position))
names = mt.varid.collect()
names_str = ",".join(names)

line = f"{mt.vep.transcript_consequences.gene_symbol[0].collect()[0]}\t{mt.locus.contig.collect()[0]}\t{position}\t{names_str}"

with open(SETLIST_FILE, "w") as f:
    f.write(line)


In [27]:
# Dx upload
bgen_file = BGEN_FILE + ".bgen"
sample_file = BGEN_FILE + ".sample"

run(
    [
        "dx",
        "upload",
        bgen_file,
        sample_file,
        ANNOTATIONS_FILE,
        SETLIST_FILE,
        "--path",
        "/Data/burden/",
    ],
    check=True,
    shell=False,
)

CompletedProcess(args=['dx', 'upload', '/opt/notebooks/gogoGPCR/data/GIPR/GIPR.bgen', '/opt/notebooks/gogoGPCR/data/GIPR/GIPR.sample', '/opt/notebooks/gogoGPCR/data/GIPR/GIPR.annotations', '/opt/notebooks/gogoGPCR/data/GIPR/GIPR.setlist', '--path', '/Data/burden/'], returncode=0)